<a href="https://colab.research.google.com/github/vinyl-bin/selenium/blob/main/selenium_change_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 46.1 MB/s 
     |████████████████████████████████| 358 kB 37.2 MB/s 
     |████████████████████████████████| 4.0 MB 29.3 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
name=['비눗방울']
category=['리뷰']

#비눗방울 리뷰
ns_address="https://search.shopping.naver.com/catalog/32312940336?query=%EB%B9%84%EB%88%97%EB%B0%A9%EC%9A%B8&NaPm=ct%3Dl3wbhe7k%7Cci%3Db1e4a573a939efb90f075e283fc583cf80e1ce6a%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D74e479a60372823701768de0d2a5983ab4cde64e"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul"

In [4]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [12]:
header = {'User-Agent': ''}
d = webdriver.Chrome('chromedriver',options=options) # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(5)

#쇼핑몰 리뷰 보기
# d.find_element_by_xpath(shoppingmall_review).click() 셀레니움에서 이 부분이 사라짐 아래의 방법으로 해야함!
element = d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul')
sleep(2)

# element=d.find_element_by_xpath(shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

In [13]:
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [15]:
d.find_element("xpath", shoppingmall_review).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

In [19]:
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료
        
    else : 
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[11]').click()
            else : next_page=d.find_elemen("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break
            

df4=add_dataframe(name_,category_,reviews,stars,cnt)
#save()

페이지 3 

1 애기랑 사이좋게 하려고 두개 샀어요 😁방울이 엄청 많이 나와요!!! 액도 좋은건지 쿠x에서 산 비눗방울 액은 방울이 아예 안만들아졌었는데 ! 😡아오 이건 속 시원히 아주 빵빵
애기랑 사이좋게 하려고 두개 샀어요 😁
방울이 엄청 많이 나와요!!!
액도 좋은건지 쿠x에서 산 비눗방울 액은
방울이 아예 안만들아졌었는데 ! 😡아오
이건 속 시원히 아주 빵빵 많이 나오네요!!!
! 👍
거기다 불 때 애가 실수로 들이켜도
! !
이거랑 비슷하게 생긴거 과거에 샀었는데
그건 애가 실수로 마시면 입안 가득 거품이였는데😰
요건 그런거 1도 없어서 !!
그리고 라이브 때 엄청난 행운 주셔서 감사해여❤️‍🔥
넘 ㅎㅎ
알라뷰 아가드👍❤️‍🔥 평점5 

2 아가 기념일 사진 셀프촬영할때마다 분위기용+포토용으로 구매했는데, 사진 촬영하면서도 예쁜 분위기 샤샤샥➰ 연출하면서 찍을 수 있어요😚🤍✨ 아가드 비눗방울 덕분에 홈스튜디오 촬영
아가 기념일 사진 셀프촬영할때마다 분위기용+포토용으로 구매했는데, 사진 촬영하면서도 예쁜 분위기 샤샤샥➰ 연출하면서 찍을 수 있어요😚🤍✨ 아가드 비눗방울 덕분에 홈스튜디오 촬영 매번 대성공👼🏻🤍💛✨✨ 촬영용 제외하고도 놀이용 또한 말모〰️ 우리 사랑둥이 아가가 아가드 비눗방울 놀이 너무 좋아해용👼🏻 선물용으로 하나 더 구매했었는데, 추가 구매 또한 계속 계속 하고 싶을 정도로 소장가치 충분해용👏🏻👏🏻💛🤍✨✨ 평점5 

3 요즘 날이 좋아서 외출하면 아기한테 비누방울 불어주고 싶어 구매했어요. 역류방지 기능이 있어 아기가 직접 불면서 놀면 더 좋겠지만 아직 어려서 엄마가 아날로그ㅎㅎ 어린시절 추억 떠
요즘 날이 좋아서 외출하면 아기한테 비누방울 불어주고 싶어 구매했어요. 역류방지 기능이 있어 아기가 직접 불면서 놀면 더 좋겠지만 아직 어려서 엄마가 아날로그ㅎㅎ 어린시절 추억 떠올리며 마구마구 불어줬습니다ㅎㅎ 아기자기 작은 비눗방울이 엄청 풍성하게 나와서 사진 찍기 좋고👍 쉽게 터지지 않아서 더 좋은 것 같아요! 화창한 날 아가드 비눗방울로 

In [20]:
df4.to_excel('sample1.xlsx')